# Exploring Hacker News Posts

Hacker News is a site started by the startup incubator Y Combinator. Users submit stories known as "posts," and others can vote and comment on them. It is a popular site for those interested in technology and startups. 

The data I will be using can be found [here](https://www.kaggle.com/hacker-news/hacker-news-posts). It is a data set of about 20,000 rows. The following are the descriptions of the columns:

`id`: The unique identifier from Hacker News for the post
`title`: The title of the post
`url`: The URL that the posts links to, if it the post has a URL
`num_points`: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
`num_comments`: The number of comments that were made on the post
`author`: The username of the person who submitted the post
`created_at`: The date and time at which the post was submitted

I am specifically interested in posts whose titles begin with `Ask HN`, where users can ask the Hacker News community a question, or `Show HN`, where users show the Hacker News community a project, product, or anything else interesting. 

I want to determine the following: 
* Do `Ask HN` or `Show HN` receive more comments on average?
* Do posts created at a certain time receive more comments on average?

In [2]:
from csv import reader

opened_hn = open('/Users/lisacondoluci/Documents/hacker_news.csv')
read_hn = reader(opened_hn)
hn = list(read_hn)
opened_hn.close()
print(hn[:4])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']]


In [3]:
headers = hn[0]
hn = hn[1:]
print(hn[:4])

[['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


Now that I have removed the header row from the data and inspected the first five rows to ensure that it is correct, I am able to separate the Hacker News posts based on their content.

In [24]:
ask_posts = []
show_posts = [] 
other_posts = []
for row in hn:
    title = row[1].lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else: 
        other_posts.append(row)
     
print('Number of Ask HN posts: ', len(ask_posts))
print('Number of Show HN posts: ', len(show_posts))
print('Number of other posts:' , len(other_posts))
print('\n')
print(ask_posts[:5])
print(show_posts[:5])


Number of Ask HN posts:  1744
Number of Show HN posts:  1162
Number of other posts: 17193


[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14'], ['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20'], ['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']]
[['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03'], ['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46'], ['11590768', 'Show

Now, I will determine which types of posts receive the highest number of comments.

In [26]:
total_ask_comments = 0
for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments
    
avg_ask_comments = total_ask_comments/len(ask_posts)
print('Average number of comments on Ask HN Posts: ', avg_ask_comments)

total_show_comments = 0
for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments/len(show_posts)
print('Average number of comments on Show HN Posts: ', avg_show_comments)

Average number of comments on Ask HN Posts:  14.038417431192661
Average number of comments on Show HN Posts:  10.31669535283993


On average, `Ask HN` posts receive more comments, at about 14, while `Show HN` posts receive only about 10 on average. 

Since `Ask HN` posts receive more comments, I'll focus my remaining analysis just on these posts. I will next determine if `Ask HN` posts created at a certain time are more likely to attract comments, so I will do the following:
* Calculate number of ask posts created in each hour of the day, along with number of comments received.
* Calculate average number of comments ask posts receive by hour created.

To do this, I will first use the `datetime` module to work with data in the `created_at` column. The `datetime.strptime()` constructor will be used to parse dates stored as strings and return datetime objects. An example is below. 

In [32]:
import datetime as dt

date_1_str = "December 24, 1984"
date_1_dt = dt.datetime.strptime(date_1_str, "%B %d, %Y")
print(date_1_dt)

1984-12-24 00:00:00
[['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29], ['5/2/2016 10:14', 1], ['8/2/2016 14:20', 3], ['10/15/2015 16:38', 17]]


In the example above:
* `%B` represents the full name of the month
* `%d` represents the two digit day
* `%Y` represents the four digit year

In [49]:
result_list = []
for post in ask_posts:
    created_at = post[6]
    num_comments = int(post[4])
    result_list.append([created_at, num_comments])
    
print('Example of results: ', result_list[:5])   
print('\n')
    
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    created_at = row[0]
    num_comments = row[1]
    created_at_dt = dt.datetime.strptime(created_at, '%m/%d/%Y %H:%M')
    hour = dt.datetime.strftime(created_at_dt, '%H')
    if hour in counts_by_hour:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += num_comments
    else: 
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = num_comments

print(counts_by_hour)
print('\n')
print(comments_by_hour)

Example of results:  [['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29], ['5/2/2016 10:14', 1], ['8/2/2016 14:20', 3], ['10/15/2015 16:38', 17]]


{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}


{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


Now that I have dictionaries created for `counts_by_hour` and `comments_by_hour`, I am able to calculate the average number of comments for posts created during each hour of the day. 

In [50]:
avg_by_hour = []
for hour in counts_by_hour:
    avg = comments_by_hour[hour]/counts_by_hour[hour]
    avg_by_hour.append([hour, avg])
    
print(avg_by_hour)

[['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


Since the current format is a bit difficult to read, I will sort the list of lists above and identify the five highest values in a format that is easier to read. I will first swap the order of the list of lists above, then sort. 

In [58]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour)
print('\n')

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print(sorted_swap)
print('\n')
print('Top 5 Hours for Ask Post Comments')

for row in sorted_swap[:5]:
    avg = row[0]
    hour = dt.datetime.strptime(row[1], '%H')
    formatted_hour = dt.datetime.strftime(hour, '%H:%M')
    print("{}: {:.2f} average comments per post".format(formatted_hour, avg))

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]


[[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [13.20183486238532, '18'], [11.46, '17'], [11.383333333333333, '01'], [11.051724137931034, '11'], [10.8, '19'], [10.25, '08'], [10.08695652173913, '05'], [9.41095890410959, '12'],

The time above is in Eastern Time in the US according to the documentation for the data set, which is the time zone I am currently in. I have found that posting at 3:00 PM ET generates the highest average number of comments per post with an average of 38.59 comments.

In order to have a higher chance of receiving comments, one should create a post at 3:00 PM ET. Closely behind is 2:00 AM, 8:00 PM, 4:00 PM, and 9:00 PM.  

Some next steps for this project include the following:
* Determine if show or ask posts receive more points on average.
* Determine if posts created at a certain time are more likely to receive more points.
* Compare your results to the average number of comments and points other posts receive.
* Use Dataquest's data science project style guide to format the project.